# Connect X

In [312]:
import numpy as np
import pandas as pd

In [9]:
# Board size
size = (6, 7)

def setup_board(size):
    # Full board has a border set to -1
    board_full = -np.ones(np.array(size) + (2, 2), dtype='int8')
    state = board_full[1:1+size[0], 1:1+size[1]]
    state[:] = 0
    return board_full, state

board_full, state = setup_board(size)
board_full

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1,  0,  0,  0,  0,  0,  0,  0, -1],
       [-1,  0,  0,  0,  0,  0,  0,  0, -1],
       [-1,  0,  0,  0,  0,  0,  0,  0, -1],
       [-1,  0,  0,  0,  0,  0,  0,  0, -1],
       [-1,  0,  0,  0,  0,  0,  0,  0, -1],
       [-1,  0,  0,  0,  0,  0,  0,  0, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1]], dtype=int8)

In [10]:
state

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int8)

In [80]:
def fill_levels(state):
    # Note: This assumes proper filling!
    return (state > 0).sum(axis=0)

def available_cols(state):
    return np.nonzero(fill_levels(state) < size[0])[0]

state[:] = np.zeros(size, dtype='int8')
assert available_cols(state).tolist() == [0, 1, 2, 3, 4, 5, 6]
state[0,3] = 1
assert available_cols(state).tolist() == [0, 1, 2, 3, 4, 5, 6]
state[0:5,1] = 2
assert available_cols(state).tolist() == [0, 1, 2, 3, 4, 5, 6]
state[0:6,size[1]-1] = 1
assert available_cols(state).tolist() == [0, 1, 2, 3, 4, 5]
state[:,:] = 1
assert available_cols(state).tolist() == []

In [81]:
def show_state(state, marks = ['S', 'O']):
    """Display the current state of the board."""

    chars = '_' + ''.join(marks)
    for row in state:
        print(" ".join(list(chars[i] for i in row)))

state[:] = np.zeros(size, dtype='int8')
state[0,3:6] = 1
state[0:4,1] = 2

show_state(state)

_ O _ S S S _
_ O _ _ _ _ _
_ O _ _ _ _ _
_ O _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _


In [82]:
steps = {
    'u': (1, 0),
    'd': (-1, 0),
    'r': (0, 1),
    'l': (0, -1),
    'ur': (1, 1),
    'dr': (-1, 1),
    'ul': (1, -1),
    'dl': (-1, -1)
}

def get_neighbours(board_full, pos):
    neighbours = {d: board_full[(step[0]+pos[0], step[1]+pos[1])] for d, step in steps.items()}
    return neighbours

pos = (1, 1)  # Note: This is actually (0, 0)
get_neighbours(board_full, pos)

{'u': 0, 'd': -1, 'r': 2, 'l': -1, 'ur': 2, 'dr': -1, 'ul': -1, 'dl': -1}

In [83]:
connect = 4

def chain_in_direction(board_full, pos, direction, role):
    """Finds number of matching discs in one direction."""
    step = steps[direction]
    for i in range(connect):
        pos = (step[0]+pos[0], step[1]+pos[1])
        x = board_full[pos]
        if x != role:
            break
    return i

state[:] = 2
show_state(state)
pos = (1, 1)
for direction in steps.keys():
    print(direction, chain_in_direction(board_full, pos, direction, 2))

O O O O O O O
O O O O O O O
O O O O O O O
O O O O O O O
O O O O O O O
O O O O O O O
u 3
d 0
r 3
l 0
ur 3
dr 0
ul 0
dl 0


In [84]:
def check_game_over(board_full, move, connect=4):

    role, pos = move
    assert board_full[pos] == 0
    results = {}
    for direction, step in steps.items():
        n = chain_in_direction(board_full, pos, direction, role)
        if n == connect - 1:
            return True
        results[direction] = n

    for d1, d2 in [('u', 'd'), ('l', 'r'), ('ul', 'dr'), ('dl', 'ur')]:
        if results[d1] + results[d2] >= connect-1:
            return True
    return False

state[:] = np.array([
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]
])
move = (2, (1, 1))
assert check_game_over(board_full, move) == False
move = (1, (1, 1))
assert check_game_over(board_full, move) == False

state[:] = np.array([
    [1, 1, 1, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]
])
move = (1, (4, 4))
assert check_game_over(board_full, move) == True
move = (1, (4, 3))
assert check_game_over(board_full, move) == True
move = (2, (4, 4))
assert check_game_over(board_full, move) == False
move = (1, (2, 1))
assert check_game_over(board_full, move) == False
move = (1, (3, 2))
assert check_game_over(board_full, move) == False
move = (1, (4, 3))
assert check_game_over(board_full, move) == True
move = (2, (4, 3))
assert check_game_over(board_full, move) == False

state[:] = np.array([
    [1, 1, 1, 0, 0, 0, 0],
    [0, 1, 1, 2, 0, 0, 0],
    [0, 0, 1, 2, 2, 0, 0],
    [0, 0, 2, 0, 2, 0, 0],
    [0, 0, 2, 0, 0, 0, 2],
    [0, 0, 0, 0, 0, 0, 0]
])
move = (2, (4, 4))
assert check_game_over(board_full, move) == False
move = (1, (4, 4))
assert check_game_over(board_full, move) == True
move = (2, (4, 6))
assert check_game_over(board_full, move) == True

In [114]:
state[:] = np.array([
    [0, 1, 1, 1, 2, 2, 1],
    [0, 0, 1, 0, 2, 1, 0],
    [0, 0, 0, 0, 2, 2, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 1, 0]
])
fill_levels(state)

array([0, 1, 2, 1, 3, 6, 1])

In [115]:
def next_available_position(state, col):
    # Note: This assumes proper filling!
    return (state[:, col] > 0).sum()

In [116]:
# Find winning moves
role = 1
for col in available_cols(state):
    pos = (next_available_position(state, col), col)
    move = (role, (pos[0]+1, pos[1]+1))
    print((role, pos), check_game_over(board_full, move))

role = 2
for col in available_cols(state):
    pos = (next_available_position(state, col), col)
    move = (role, (pos[0]+1, pos[1]+1))
    print((role, pos), check_game_over(board_full, move))

(1, (0, 0)) True
(1, (1, 1)) False
(1, (2, 2)) False
(1, (1, 3)) False
(1, (3, 4)) False
(1, (1, 6)) False
(2, (0, 0)) False
(2, (1, 1)) False
(2, (2, 2)) False
(2, (1, 3)) False
(2, (3, 4)) True
(2, (1, 6)) False


In [125]:
fl = fill_levels(state)
fl

array([0, 1, 2, 1, 3, 6, 1])

In [207]:
size[0]

6

In [206]:
fl

array([0, 1, 2, 1, 3, 6, 1])

In [238]:
cols = np.arange(1,size[1]+1)
rows = [ for i in range(4)]
rows

[(array([1, 2, 3, 2, 4, 7, 2]), array([1, 2, 3, 4, 5, 6, 7])),
 (array([0, 1, 2, 1, 3, 6, 1]), array([1, 2, 3, 4, 5, 6, 7])),
 (array([0, 0, 1, 0, 2, 5, 0]), array([1, 2, 3, 4, 5, 6, 7])),
 (array([0, 0, 0, 0, 1, 4, 0]), array([1, 2, 3, 4, 5, 6, 7]))]

In [261]:
fl

array([0, 1, 2, 1, 3, 6, 1])

In [264]:
rows = np.empty((4, size[1]), dtype='int8')
for i in range(4):
    rows[i] = (fl-i).clip(-1,)
rows

array([[ 0,  1,  2,  1,  3,  6,  1],
       [-1,  0,  1,  0,  2,  5,  0],
       [-1, -1,  0, -1,  1,  4, -1],
       [-1, -1, -1, -1,  0,  3, -1]], dtype=int8)

In [265]:
cols = np.empty((4, size[1]), dtype='int8')
cols[:] = np.arange(1,size[1]+1)
cols

array([[1, 2, 3, 4, 5, 6, 7],
       [1, 2, 3, 4, 5, 6, 7],
       [1, 2, 3, 4, 5, 6, 7],
       [1, 2, 3, 4, 5, 6, 7]], dtype=int8)

In [268]:
board_full

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1,  0,  1,  1,  1,  2,  2,  1, -1],
       [-1,  0,  0,  1,  0,  2,  1,  0, -1],
       [-1,  0,  0,  0,  0,  2,  2,  0, -1],
       [-1,  0,  0,  0,  0,  0,  2,  0, -1],
       [-1,  0,  0,  0,  0,  0,  2,  0, -1],
       [-1,  0,  0,  0,  0,  0,  1,  0, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1]], dtype=int8)

In [267]:
board_full[(rows, cols)]

array([[-1,  1,  1,  1,  2,  1,  1],
       [-1, -1,  1, -1,  2,  2, -1],
       [-1, -1, -1, -1,  2,  2, -1],
       [-1, -1, -1, -1, -1,  2, -1]], dtype=int8)

In [278]:
m_level = int(np.median(fl))
m_level

1

In [281]:
state_key = np.empty((5, size[1]), dtype='int8')
state_key[0, :] = (fl - m_level).clip(-1, 2)
state_key[1:, :] = board_full[(rows, cols)]
state_key

array([[-1,  0,  1,  0,  2,  2,  0],
       [-1,  1,  1,  1,  2,  1,  1],
       [-1, -1,  1, -1,  2,  2, -1],
       [-1, -1, -1, -1,  2,  2, -1],
       [-1, -1, -1, -1, -1,  2, -1]], dtype=int8)

In [275]:
4**16

4294967296

In [276]:
4**12

16777216

In [283]:
(16*1294967296)/1e6

20719.476736

In [284]:
3**16

43046721

### Find all possible (valid) combinations of states for 4x4 segment

In [311]:
from itertools import product

combinations = np.array(list(product(range(5), repeat=4))).sum(axis=1)
assert len(combinations) == 625

In [334]:
n_discs = pd.Series(combinations).value_counts().sort_index().rename('No. of boards')
n_discs.index.name = 'No. of discs'
summary = pd.concat([
    n_discs, 
    pd.Series(2**n_discs.index.values, index=n_discs.index, name='Disc combinations')
], axis=1)
summary['Total combinations'] = summary['No. of boards'] * summary['Disc combinations']
summary

,No. of boards,Disc combinations,Total combinations
No. of discs,,,
0,1,1,1
1,4,2,8
2,10,4,40
3,20,8,160
4,35,16,560
5,52,32,1664
6,68,64,4352
7,80,128,10240
8,85,256,21760


In [335]:
summary['Total combinations'].sum()

923521

### Generate a compact, hashable state representation

In [523]:
a1 = np.array([64, 16, 4, 1])
a2 = np.array([16777216, 65536, 256, 1])

def generate_state_key_uint32(grid):
    """Convert 4x4 int8 array to int64 value.
    """
    assert grid.dtype == 'int8'
    assert grid.shape == (4, 4)
    return (np.sum(grid*a1, axis=1)*a2).sum().astype('uint32')

grid = 3*np.ones((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 2**32-1
assert generate_state_key_uint32(grid).dtype == 'uint32'
assert (generate_state_key_uint32(grid) + 1).dtype == 'int64'
grid = np.zeros((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 0
grid[:, 3] = 1
assert generate_state_key_uint32(grid) == 1 + 256 + 256**2 + 256**3
grid = np.ones((4,3), dtype='int8') 
try:
    generate_state_key_uint32(grid)
except AssertionError:
    pass

grid = np.random.choice([0, 1, 2], size=16).astype('int8').reshape((4,4))
print(grid)
generate_state_key_uint32(grid)

[[0 0 1 2]
 [0 0 2 2]
 [0 0 2 2]
 [0 0 1 0]]


101321220

In [527]:
# sum([30.9, 31, 31.1, 28.9])/4 == 30.5
%timeit generate_state_key_uint32(grid)

28.9 µs ± 566 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [529]:
a1 = np.array([6, 4, 2, 0])
a2 = np.array([24, 16, 8, 0])

def generate_state_key_uint32(grid):
    """Convert 4x4 int8 array to int64 value.
    """
    assert grid.dtype == 'int8'
    return np.sum(np.sum(grid << a1, axis=1) << a2).astype('uint32')

grid = 3*np.ones((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 2**32-1
assert generate_state_key_uint32(grid).dtype == 'uint32'
assert (generate_state_key_uint32(grid) + 1).dtype == 'int64'
grid = np.zeros((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 0
grid[:, 3] = 1
assert generate_state_key_uint32(grid) == 1 + 256 + 256**2 + 256**3
grid = np.ones((4,3), dtype='int8') 
try:
    generate_state_key_uint32(grid)
except ValueError:
    pass

grid = np.random.choice([0, 1, 2], size=16).astype('int8').reshape((4,4))
print(grid)
generate_state_key_uint32(grid)

[[2 0 2 0]
 [1 0 1 2]
 [2 1 0 1]
 [2 0 2 0]]


2286326152

In [555]:
# sum([31.2, 31.9, 31.8, 32.4])/4 == 31.8
%timeit generate_state_key_uint32(grid)

31.8 µs ± 827 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [562]:
a = np.array([1, 2, 3, 0], dtype='int8')

v = np.sum(a << np.array([6, 4, 2, 0])).astype('uint32')

v

108

In [563]:
(v >> 6) & 3, (v >> 4) & 3, (v >> 2) & 3, v & 3

(1, 2, 3, 0)

In [566]:
np.array([v >> 6, v >> 4, v >> 2, v], dtype='int8') & 3

array([1, 2, 3, 0], dtype=int8)